In [1]:
def f1():
    x = 10
    
    def f2():
        nonlocal x
        x = 20
        print('x in f2 is', x)
    
    f2()
    print('x in f1 is', x)

x = 5
f1()
print('Global x:', x)

x in f2 is 20
x in f1 is 20
Global x: 5


In [7]:
def make_withdraw(balance):
    def withdraw(amount):
        nonlocal balance
        if amount > balance:
            return 'Insufficient funds'
        balance = balance - amount
        return balance
    return withdraw

wd = make_withdraw(20)
wd(5)

15

In [2]:
def f1():
    x = 10
    
    def f2():
        x = 20
        print('x in f2 is', x)
    
    f2()
    print('x in f1 is', x)

x = 5
f1()
print('Global x:', x)

x in f2 is 20
x in f1 is 10
Global x: 5


In [5]:
def f1():
    x = 10
    
    def f2():
        
        print('x in f2 is', x)
    
    f2()
    print('x in f1 is', x)

x = 5
f1()
print('Global x:', x)

x in f2 is 10
x in f1 is 10
Global x: 5


In [1]:
def dictionary():
    
    records = []
    def getitem(key):
        matches = [r for r in records if r[0] == key]
        if len(matches) == 1:
            key, value = matches[0]
            return value
    def setitem(key, value):
        nonlocal records
        non_matches = [r for r in records if r[0] != key]
        records = non_matches + [[key, value]]
    def dispatch(message, key=None, value=None):
        if message == 'getitem':
            return getitem(key)
        elif message == 'setitem':
            setitem(key, value)
    return dispatch

In [5]:
my_dict = dictionary()

In [6]:
my_dict('setitem', key='LIU', value=23)
my_dict('setitem', key='WANG', value=21)

In [7]:
my_dict('getitem', key='LIU')

23

In [9]:
def account(initial_balance):
    def deposit(amount):
        dispatch['balance'] += amount
        return dispatch['balance']
    def withdraw(amount):
        if amount > dispatch['balance']:
            return 'Insufficient funds'
        dispatch['balance'] -= amount
        return dispatch['balance']
    dispatch = {'deposit': deposit,
                'withdraw': withdraw,
                'balance': initial_balance
                }
    return dispatch

In [10]:
def withdraw(account, amount):
    return account['withdraw'](amount)
def deposit(account, amount):
    return account['deposit'](amount)
def check_balance(account):
    return account['balance']

In [11]:
a = account(20)
deposit(a, 5)
withdraw(a, 17)
check_balance(a)

8

In [1]:
from operator import add, sub
from operator import mul, truediv

In [2]:
def make_ternary_constarint(a, b, c, ab, ca, cb):
    '''The constraint that ab(a, b)=c and ca(c, a)=b and cb(c, b)=a'''
    def new_value():
        av, bv, cv = [connector['has_val']() for  connector in (a, b, c)]
        if av and bv:
            c['set_val'](constraint, ab(a['val'], b['val']))
        elif av and cv:
            b['set_val'](constraint, ca(c['val'], a['val']))
        elif bv and cv:
            a['set_val'](constraint, cb(c['val'], b['val']))
    
    def forget_value():
        for connector in (a, b, c):
            connector['forget'](constraint)
    
    constraint = {'new_val': new_value, 'forget': forget_value}
    for connector in (a, b, c):
        connector['connect'](constraint)
    return constraint

In [ ]:
def multiplier(a, b, c):
    '''The constraint that a * b = c'''
    return make_ternary_constarint(a, b, c, mul, truediv, truediv)
def adder(a, b, c):
    '''The constraint that a + b = c'''
    return make_ternary_constarint(a, b, c, add, sub, sub)
def constant(connector, value):
    '''The constraint that connector = value'''
    constraint = {}
    connector['set_val'](constraint, value)
    return constraint
def inform_all_except(source, message, constraints):
    '''inform all constraints of the message, expect source'''
    for c in constraints:
        if c != source:
            c[message]()
def connector(name=None):
    '''A connector between constraints'''
    informant = None
    constraints = []
    
    def set_value(source, value):
        nonlocal informant
        val = connector['val']
        if val is None:
            informant, connector['val'] = source, value
            if name is not None:
                print(name, '=', value)
            inform_all_except(source, 'new_val', constraints)
        else:
            if val != value:
                print('Constradiction detected:', val, 'vs', value)
    
    def forget_value(source):
        nonlocal informant
        if informant == source:
            informant, connector['val'] = None, None
            if name is not None:
                print(name, 'is forgotten')
            inform_all_except(source, 'forget', constraints)
    
    connector = {'val':  None,
                 'set_val':  set_value,
                 'forget':  forget_value,
                 'has_val': lambda: connector['val'] is not None,
                 'connect': lambda source: constraints.append(source)
                 }
    return connector
    
celsius = connector('Celsius')
fahrenheit = connector('Fahrenheit')

def convert(c, f):
    u, v, w, x, y = [connector() for _ in range(5)]
    multiplier(c, w, u)
    multiplier(v, x, u)
    adder(v, y, f)
    constant(w, 9)
    constant(x, 5)
    constant(y, 32)
    
convert(celsius, fahrenheit)

In [21]:
celsius['set_val']('user', 25)

Celsius = 25
Fahrenheit = 77.0
